# List of contents
### 01. Importing libraries and data
### 02. Grouping and aggregating
### 03. Creating flag columns with aggregated variables
### 04. Clean and export data

# 01. Importing libraries and data

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Import data
path = r'C:\Users\Lawrence\Documents\Career Foundry Projects\Data Immersion\Achievement #4\2023 Instacart Basket Analysis'
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_derived.pkl'))

# 02. Grouping and aggregating

In [3]:
# Step 2
# Find the aggregated mean of the order_number column grouped by department_id
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


In [4]:
# Step 3 
# Compare these results to the subset of 1,000,000 orders
subset = ords_prods_merge[:1000000]
subset.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
4,18.825780
7,17.472355
13,17.993423
14,19.246334
16,19.463012
17,11.294069
19,19.305237
20,17.599636


The results for the full dataframe contain information about orders from all 22 departments, while the results for the subset contain information about orders from only 8 departments. The means are also different. The means for the full dataframe are smaller for most department IDs (4, 7, 13, 14, 16, and 19), but larger for a few (17 and 20).

# 03. Creating flag columns with aggregated variables

In [5]:
# Step 4
# Create a loyalty flag column for existing customers
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

C:\Users\Lawrence\AppData\Local\Temp\ipykernel_23532\2136683801.py:3: FutureWarning: The provided callable <function amax at 0x00000222748198A0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)


In [6]:
ords_prods_merge.head(30)

,order_id,user_id,order_number,orders_day_of_the_week,order_hour_of_day,days_since_prior_order_number,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Slowest days,Average orders,10
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Slowest days,Most orders,10
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Slowest days,Average orders,10
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Slowest days,Most orders,10
5,3367565,1,6,2,7,19.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10
6,550135,1,7,1,9,20.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Busiest days,Most orders,10
7,3108588,1,8,1,14,14.0,196,2,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Busiest days,Most orders,10
8,2295261,1,9,1,16,0.0,196,4,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Busiest days,Most orders,10
9,2550362,1,10,4,8,30.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Slowest days,Average orders,10


In [7]:
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

C:\Users\Lawrence\AppData\Local\Temp\ipykernel_23532\3647449420.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Loyal customer' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'


In [8]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] > 10)
                     & (ords_prods_merge['max_order'] <= 40), 'loyalty_flag'] = 'Regular customer'

In [9]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [10]:
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

In [11]:
# Step 5
# Check basic statistics of the product prices for each loyalty category
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

In [12]:
# Step 6
# Create a spending flag for each user based on the average price across all their orders
ords_prods_merge['avg_price_of_user'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

C:\Users\Lawrence\AppData\Local\Temp\ipykernel_23532\3696022780.py:3: FutureWarning: The provided callable <function mean at 0x000002227481A5C0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ords_prods_merge['avg_price_of_user'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)


In [13]:
ords_prods_merge['avg_price_of_user'].head(10)

0    6.367797
1    6.367797
2    6.367797
3    6.367797
4    6.367797
5    6.367797
6    6.367797
7    6.367797
8    6.367797
9    6.367797
Name: avg_price_of_user, dtype: float64

In [14]:
ords_prods_merge.loc[ords_prods_merge['avg_price_of_user'] < 10, 'spending_level'] = 'Low spender.'

C:\Users\Lawrence\AppData\Local\Temp\ipykernel_23532\3880967531.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Low spender.' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ords_prods_merge.loc[ords_prods_merge['avg_price_of_user'] < 10, 'spending_level'] = 'Low spender.'


In [15]:
ords_prods_merge.loc[ords_prods_merge['avg_price_of_user'] >= 10, 'spending_level'] = 'High spender.'

In [16]:
ords_prods_merge['spending_level'].value_counts(dropna = False)

spending_level
Low spender.     31770614
High spender.      634245
Name: count, dtype: int64

In [17]:
# Step 7
# Create an order frequency flag that marks the regularity of a user's ordering behavior
# according to the median in the "days_since_prior_order" column
ords_prods_merge['median_days_since_prior_order'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order_number'].transform(np.median)

C:\Users\Lawrence\AppData\Local\Temp\ipykernel_23532\3020658915.py:4: FutureWarning: The provided callable <function median at 0x000002227495BEC0> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  ords_prods_merge['median_days_since_prior_order'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order_number'].transform(np.median)


In [18]:
ords_prods_merge.loc[ords_prods_merge['median_days_since_prior_order'] > 20, 'ordering_regularity'] = 'Non-frequent customer.'

C:\Users\Lawrence\AppData\Local\Temp\ipykernel_23532\680299955.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Non-frequent customer.' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ords_prods_merge.loc[ords_prods_merge['median_days_since_prior_order'] > 20, 'ordering_regularity'] = 'Non-frequent customer.'


In [19]:
ords_prods_merge.loc[(ords_prods_merge['median_days_since_prior_order'] <= 20)
                     & (ords_prods_merge['median_days_since_prior_order'] > 10), 'ordering_regularity'] = 'Regular customer.'

In [20]:
ords_prods_merge.loc[ords_prods_merge['median_days_since_prior_order'] <= 10, 'ordering_regularity'] = 'Frequent customer.'

# 04. Clean and export data

In [21]:
ords_prods_merge['ordering_regularity'].value_counts(dropna = False)

ordering_regularity
Frequent customer.        21559853
Regular customer.          7208564
Non-frequent customer.     3636437
NaN                              5
Name: count, dtype: int64

In [22]:
# Investigate the 5 counts of NaN
ords_prods_merge.loc[ords_prods_merge['ordering_regularity'].isna()]

,order_id,user_id,order_number,orders_day_of_the_week,order_hour_of_day,days_since_prior_order_number,product_id,add_to_cart_order,reordered,product_name,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price_of_user,spending_level,median_days_since_prior_order,ordering_regularity
13645692,895835,159838,1,0,17,NaN,10749,3,0,Organic Red Bell Pepper,...,Mid-range product,Busiest day,Busiest days,Average orders,1,New customer,7.42,Low spender.,NaN,NaN
17251990,895835,159838,1,0,17,NaN,33401,6,0,Goat Cheese Crumbles,...,Mid-range product,Busiest day,Busiest days,Average orders,1,New customer,7.42,Low spender.,NaN,NaN
17622767,895835,159838,1,0,17,NaN,23695,2,0,California Veggie Burger,...,Low-range product,Busiest day,Busiest days,Average orders,1,New customer,7.42,Low spender.,NaN,NaN
24138593,895835,159838,1,0,17,NaN,21334,5,0,Organic Peeled Garlic,...,Mid-range product,Busiest day,Busiest days,Average orders,1,New customer,7.42,Low spender.,NaN,NaN
25880002,895835,159838,1,0,17,NaN,22198,1,0,4X Ultra Concentrated Natural Laundry Detergen...,...,Low-range product,Busiest day,Busiest days,Average orders,1,New customer,7.42,Low spender.,NaN,NaN


In [23]:
ords_prods_merge.loc[ords_prods_merge['max_order'] == 1].head(30)

,order_id,user_id,order_number,orders_day_of_the_week,order_hour_of_day,days_since_prior_order_number,product_id,add_to_cart_order,reordered,product_name,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price_of_user,spending_level,median_days_since_prior_order,ordering_regularity
13645692,895835,159838,1,0,17,NaN,10749,3,0,Organic Red Bell Pepper,...,Mid-range product,Busiest day,Busiest days,Average orders,1,New customer,7.42,Low spender.,NaN,NaN
17251990,895835,159838,1,0,17,NaN,33401,6,0,Goat Cheese Crumbles,...,Mid-range product,Busiest day,Busiest days,Average orders,1,New customer,7.42,Low spender.,NaN,NaN
17622767,895835,159838,1,0,17,NaN,23695,2,0,California Veggie Burger,...,Low-range product,Busiest day,Busiest days,Average orders,1,New customer,7.42,Low spender.,NaN,NaN
24138593,895835,159838,1,0,17,NaN,21334,5,0,Organic Peeled Garlic,...,Mid-range product,Busiest day,Busiest days,Average orders,1,New customer,7.42,Low spender.,NaN,NaN
25880002,895835,159838,1,0,17,NaN,22198,1,0,4X Ultra Concentrated Natural Laundry Detergen...,...,Low-range product,Busiest day,Busiest days,Average orders,1,New customer,7.42,Low spender.,NaN,NaN


Identified 5 counts of customers with one-and-done orders. These cases would have a null value for median_days_since_prior order.

In [24]:
# Data cleaning imported from Exercise 4.9
# Mark rows with outlier prices as missing
ords_prods_merge.loc[ords_prods_merge['prices'] > 100, 'prices'] = np.nan
ords_prods_merge['prices'].max()

25.0

In [25]:
# Export data
ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_aggregated.pkl'))